In [ ]:
# Tagging et extraction : un des plus importants cas d'usage des fonctions d'OpenAI
# Tagging : Extraire des données structurées d'un texte (non structuré) : 
# ex : sentiment (positif), langue (espagnol), etc.
# Extraction : sortir l'ensemble des informations relatives à un objet défini 
# ex : Bibliographie de l'ensemble des sources citées dans un article, avec nom des auteurs, date, etc.

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
# On veut créer une fonction tagging, capable de repérer le sentiment et la langue du texte en input
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [ ]:
# On retrouve la structuration JSONtypique des fonctions OpenAI
convert_pydantic_to_openai_function(Tagging)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(temperature=0)

In [ ]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [ ]:
# On veut forcer le modèle à tagger les inputs, d'où "function_call"
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [ ]:
tagging_chain = prompt | model_with_functions

In [ ]:
tagging_chain.invoke({"input": "I love langchain"})

In [ ]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

In [ ]:
# L'output de cette chaîne est un JSON, on va donc mettre en bout de chaîne un maillon qui lit les JSON
# et ne récupère que les informations taggées
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [ ]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [ ]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

In [ ]:
# Extraction
# Comme le tagging mais on extrait plusieurs informations

In [ ]:
# On crée une classe permettant de retrouver une personne, avec si possible son âge (en option)
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [ ]:
# On veut une liste des classes ci-dessus trouvées dans les textes, c'est cette liste qu'on va transformer en fonction
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [ ]:
convert_pydantic_to_openai_function(Information)

In [ ]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [ ]:
# En résultat ci-dessous, le modèle ne connaît pas l'âge de Martha et gère mal cette ignorance
extraction_model.invoke("Joe is 30, his mom is Martha")

In [ ]:
# On va gérer le comportement du modèle en cas d'ignorance sur l'âge grâce au prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [ ]:
extraction_chain = prompt | extraction_model

In [ ]:
# Ici, l'âge de Martha n'est pas mentionné
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

In [ ]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

In [ ]:
# Pour se débarrasser du nom de la liste ("people") on utilise un maillon
# qui reconnaît et vire les noms de listes (keys)
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [ ]:
# On ne récupère que les JSON structurés appartenant à la catégorie de la clé prédéfinie dans la chaîne
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

In [ ]:
# Sur un texte plus long
# Exemple : on extrait des infos d'une sous-partie d'un article de blog

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [ ]:
doc = documents[0]

In [ ]:
page_content = doc.page_content[:10000]

In [ ]:
# le document est long et pas très structuré
print(page_content[:1000])

In [ ]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [ ]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [ ]:
tagging_chain.invoke({"input": page_content})

In [ ]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [ ]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [ ]:
# résultat ci-dessous décevant : le prompt ne demande pas explicitement de faire une biblio
extraction_chain.invoke({"input": page_content})

In [ ]:
# On améliore le prompt pour que le modèle fasse réellement une biblio
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [ ]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [ ]:
extraction_chain.invoke({"input": page_content})

In [ ]:
extraction_chain.invoke({"input": "hi"})

In [ ]:
# Si on part d'un très long article, il faut le séparer en morceaux pour le faire passer dans le LLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [ ]:
# On découpe en morceaux l'article long
splits = text_splitter.split_text(doc.page_content)

In [ ]:
len(splits)

In [ ]:
# On va découper l'article en morceaux, faire l'extraction d'infos dans chaque morceau, et rabouter
# La fonction ci-dessous raboutera les différentes listes d'infos extraites
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [ ]:
flatten([[1, 2], [3, 4]])

In [ ]:
print(splits[0])

In [ ]:
# Lorsque le premier maillon de la chaîne est une fonction, il faut utiliser le module ci-dessous
# pour convertir la liste de textes en liste d'inputs compatibles avec le prompt
from langchain.schema.runnable import RunnableLambda

In [ ]:
# Pre-processing function
# La variable d'entrée est le texte entier, qui est découpé, puis mis en forme correctement
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [ ]:
prep.invoke("hi")

In [ ]:
# utilisation de "map" pour appliquer la chaîne à une liste d'inputs au bon format
# Le mapping est partiellement parallélisé (5 appels en même temps)
chain = prep | extraction_chain.map() | flatten

In [ ]:
chain.invoke(doc.page_content)